In [1]:
import pandas as pd
import numpy as np
import requests
import os
from IPython.display import display
import urllib.request
import time
import shutil

import random
random.seed(1)

In [2]:
if not os.path.exists('ml-20m'):

    import urllib.request
    urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-20m.zip', 'ml-20m.zip')

    import zipfile
    with zipfile.ZipFile('ml-20m.zip',"r") as zip_ref:
        zip_ref.extractall()
        
    os.remove('ml-20m.zip')

In [3]:
movies = pd.read_csv('ml-20m/movies.csv')
movies.dropna(inplace=True)
movies = movies[(movies['genres'] != '(no genres listed)') & (movies['genres'].map(lambda r: 'IMAX' not in r))]

links = pd.read_csv('ml-20m/links.csv')
links.dropna(inplace=True)
links.tmdbId = links.tmdbId.astype(int)

movies = pd.merge(movies, links, how='inner', on=['movieId'])
movies.dropna(inplace=True)
movies.head()

#links.head()
#movies.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357
4,5,Father of the Bride Part II (1995),Comedy,113041,11862


In [4]:
from collections import Set

def calc_existing_genres(movies):
    genres_set = set()
    for genre_list in movies['genres']:
        for genre in genre_list.split('|'):
            genres_set.add(genre)
    return genres_set
        
calc_existing_genres(movies)

{'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [5]:
if not os.path.exists('extended_movie_data.csv'):
    movies['release_year'] = ''
    movies['poster_url'] = ''
    movies.to_csv('extended_movie_data.csv', sep=';', index=False)
    
movies = pd.read_csv('extended_movie_data.csv', sep=';', keep_default_na=False)
movies.head()

,movieId,title,genres,imdbId,tmdbId,release_year,poster_url,language
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862,1995,https://image.tmdb.org/t/p/w300/rhIRbceoE9lR4v...,en
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844,1995,https://image.tmdb.org/t/p/w300/vgpXmVaVyUL7GG...,en
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602,1995,https://image.tmdb.org/t/p/w300/6ksm1sjKMFLbO7...,en
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357,1995,https://image.tmdb.org/t/p/w300/16XOMpEaLWkrcP...,en
4,5,Father of the Bride Part II (1995),Comedy,113041,11862,1995,https://image.tmdb.org/t/p/w300/e64sOI48hQXyru...,en


In [6]:
from dateutil.parser import parse

tmdb_movie_url = 'https://api.themoviedb.org/3/movie/{}?api_key=c4d5a44a2b7b36e4d40d0d1e713a8fee'
poster_base_url = 'https://image.tmdb.org/t/p/w300{}'


def get_more_movie_data(tmdbId): 
    try:
        response = requests.get(tmdb_movie_url.format(tmdbId))
    except:
        print('Data {} couldn\'t be retrieved'.format(tmdbId))
        return None
    
    tmdb_movie_json = response.json()
    
    year = 'not found'
    language = 'not found'
    poster_url = 'not found'
    
    try:
        date_string = tmdb_movie_json['release_date']
    
        try:
            dt = parse(date_string)
            year = dt.year
        except:
            print(tmdb_movie_json)
            print('Date {} can\'t be parsed'.format(date_string))
    except:
        pass
    
    try:
        language = tmdb_movie_json['original_language']
    except:
        pass
    
    try:
        if tmdb_movie_json['poster_path'] is None:
            poster_url = None
        else:
            poster_url = poster_base_url.format(tmdb_movie_json['poster_path'])
    except:
        pass
    
    return year, poster_url, language
    

In [7]:
count = 0

for index, row in movies.iterrows():
    if row['release_year'] == '':
        time.sleep(0.5)
        extended_movie_data = get_more_movie_data(row.tmdbId)
        
        if extended_movie_data is None:
            movies.iat[index, movies.columns.get_loc('release_year')] = 'not found'
            movies.iat[index, movies.columns.get_loc('poster_url')] = 'not found'
            movies.iat[index, movies.columns.get_loc('language')] = 'not found'
        else:
            release_year, poster_url, language = extended_movie_data
            
            movies.iat[index, movies.columns.get_loc('release_year')] = release_year
            movies.iat[index, movies.columns.get_loc('poster_url')] = poster_url
            movies.iat[index, movies.columns.get_loc('language')] = language
            
        count += 1
        if count % 20 == 0:
            print('{} movies extended'.format(count))
            movies.to_csv('extended_movie_data.csv', sep=';', index=False)
 

movies.to_csv('extended_movie_data.csv', sep=';', index=False)

In [22]:
def create_or_read_movies():
    if not os.path.exists('extended_movie_data_with_local_files.csv'):
        movies['local_poster_file'] = ''
        movies.to_csv('extended_movie_data_with_local_files.csv', sep=';', index=False)

    movies = pd.read_csv('extended_movie_data_with_local_files.csv', sep=';', keep_default_na=False)
    movies.head()
    return movies

movies = create_or_read_movies()
display(movies.head())
movies.shape


,movieId,title,genres,imdbId,tmdbId,release_year,poster_url,language,local_poster_file
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862,1995,https://image.tmdb.org/t/p/w300/rhIRbceoE9lR4v...,en,1.jpg
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844,1995,https://image.tmdb.org/t/p/w300/vgpXmVaVyUL7GG...,en,2.jpg
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602,1995,https://image.tmdb.org/t/p/w300/6ksm1sjKMFLbO7...,en,3.jpg
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357,1995,https://image.tmdb.org/t/p/w300/16XOMpEaLWkrcP...,en,4.jpg
4,5,Father of the Bride Part II (1995),Comedy,113041,11862,1995,https://image.tmdb.org/t/p/w300/e64sOI48hQXyru...,en,5.jpg


In [26]:
import importlib
import helpers
importlib.reload(helpers)

movies = create_or_read_movies()
filtered_movies = helpers.filter_movies(movies)
filtered_movies.shape

NameError: name 'create_or_read_movies' is not defined

In [24]:
def retrieve_poster(poster_url):
    poster_response = None
    tries = 0
    while poster_response is None:
        try:
            poster_response = requests.get(poster_url, stream=True)
        except:
            if tries >= 10:
                print('Poster {} couldn\'t be retrieved'.format(poster_url))
                break
            else:
                time.sleep(5)
                continue
    return poster_response

movies = create_or_read_movies()
filtered_movies = filter_movies(movies)       
count = 0
for index, row in filtered_movies.iterrows():
    poster_url = row.poster_url
    if row.local_poster_file == '' and poster_url != '' and poster_url != 'not found':
        poster_response = retrieve_poster(poster_url) 
        local_poster_file = str(row.movieId) + '.' + poster_url.split('.')[-1]
        poster_save_path = os.path.join('posters', local_poster_file)
        with open(poster_save_path, 'wb') as out_file:
            shutil.copyfileobj(poster_response.raw, out_file)

        movies.iat[index, movies.columns.get_loc('local_poster_file')] = local_poster_file 

        count += 1
        if count % 10 == 0:
            print('{} posters saved'.format(count))
            movies.to_csv('extended_movie_data_with_local_files.csv', sep=';', index=False)
    
    elif row.local_poster_file != '':
        count += 1
        if count % 100 == 0:
            print('{} posters saved'.format(count))

movies.to_csv('extended_movie_data_with_local_files.csv', sep=';', index=False)            
        
#movies.iat[index, movies.columns.get_loc('local_poster_file')] = local_poster_file         

100 posters saved
200 posters saved
300 posters saved
400 posters saved
500 posters saved
600 posters saved
700 posters saved
800 posters saved
https://image.tmdb.org/t/p/w300/t4M6RrwBQKTqbnG9vqG339G7FCC.jpg
https://image.tmdb.org/t/p/w300/jrjzPjKnpJq99GPePKf64k7Fzqh.jpg
https://image.tmdb.org/t/p/w300/8csiFTFkikhW7XgonmWhibH3tRo.jpg
https://image.tmdb.org/t/p/w300/tdTCGOxbdi1OC5kWwK4TsVZIvJZ.jpg
https://image.tmdb.org/t/p/w300/fIynQQgyxWCs0kNJ7YHKTRNwaZI.jpg
https://image.tmdb.org/t/p/w300/nwPZzUywExiwOZ8zyKrqGyoguuX.jpg
https://image.tmdb.org/t/p/w300/iQ8xiJmLsS0EoH3DhX0YdrbmRu3.jpg
https://image.tmdb.org/t/p/w300/vixSXefeKdPSimIoTktc6TnoWpz.jpg
https://image.tmdb.org/t/p/w300/pnnfJMRaL3UL1KlzzskFb6Lurlk.jpg
https://image.tmdb.org/t/p/w300/1G9PZLULbbIrb74H6BdhQGplCxf.jpg
820 posters saved
https://image.tmdb.org/t/p/w300/3uixF2pHa0whTKcYmenDrArX0II.jpg
https://image.tmdb.org/t/p/w300/2ek9pyqNFQbtwtXptRZxsIGkOuN.jpg
https://image.tmdb.org/t/p/w300/ajcYe8jGil6NSyUQ6KkMIcNUXBI.jpg
https:

https://image.tmdb.org/t/p/w300/pEg0tYYhSfsyjLIYpvD9lCWLhcH.jpg
https://image.tmdb.org/t/p/w300/b9FJ7sYpyU8RWrZesfmAJZRoKIm.jpg
https://image.tmdb.org/t/p/w300/x5QUB2aRVWCsdJjHEoAXfWxhReR.jpg
https://image.tmdb.org/t/p/w300/g20GsTJ3gDVEt17Z13vXzjJ7RON.jpg
https://image.tmdb.org/t/p/w300/crycJDHRvpQyZeQyRYtIZKIpNzK.jpg
https://image.tmdb.org/t/p/w300/1ELrosU04wEbfmxK9wLBjdEECqm.jpg
940 posters saved
https://image.tmdb.org/t/p/w300/i49r4ycCQAoGh2esfnL57PWGNh.jpg
https://image.tmdb.org/t/p/w300/wJYpcNbeRqEqPM52nIzBiPgCsgc.jpg
https://image.tmdb.org/t/p/w300/VqGFoorATbqU41Pmhz5VwCrXM4.jpg
https://image.tmdb.org/t/p/w300/yVgqD24O6aWeRjgNQrcij5tbglx.jpg
https://image.tmdb.org/t/p/w300/8iy8JlIgLPACRR32wzTJBpZ3iMk.jpg
https://image.tmdb.org/t/p/w300/zxjJ2IxhilZLzl8eXjj7S0apZ7k.jpg
https://image.tmdb.org/t/p/w300/5pfHUueAIYMUICAOAGpmsJPIwLz.jpg
https://image.tmdb.org/t/p/w300/u0690Aor60ZM7Zhg6rwUD1KvJOr.jpg
https://image.tmdb.org/t/p/w300/d0YVvWpirA43wvHPwV5m8ZNJsFA.jpg
https://image.tmdb.org/t

https://image.tmdb.org/t/p/w300/dRV0aqhalPHVRwHKHTsj36QVhTR.jpg
1060 posters saved
https://image.tmdb.org/t/p/w300/mIMOvooSOK2RI33t9uAfibOOWbv.jpg
https://image.tmdb.org/t/p/w300/3neHR1oPnC0OpcYhLn0TEdSHoEp.jpg
https://image.tmdb.org/t/p/w300/bRaQtgsKSXAL0ea8MZ26bIyJs1T.jpg
https://image.tmdb.org/t/p/w300/3noBOQroTjMWAYhlku95Y06v5C5.jpg
https://image.tmdb.org/t/p/w300/k4d5K7pH7Ie7Tg7hGBfkt22Hj1P.jpg
https://image.tmdb.org/t/p/w300/nnVA3Nmry9furen0NDmJa0QTn2F.jpg
https://image.tmdb.org/t/p/w300/5Np88iigCfnXvMlMKmCcbPVIbYQ.jpg
https://image.tmdb.org/t/p/w300/1AKAfnT3nm7asU9qyNT6gtS1vbe.jpg
https://image.tmdb.org/t/p/w300/2sVJSEZRczNiz9YIbnE5OVIkaHo.jpg
https://image.tmdb.org/t/p/w300/xCIPFFTSr84SxiU8BaEmpVxDCNZ.jpg
1070 posters saved
https://image.tmdb.org/t/p/w300/dgRX7epN8IMulmKSLY2klRYsz3C.jpg
https://image.tmdb.org/t/p/w300/2kWpjQ3O6DZywdIoPlvXCk3T91P.jpg
https://image.tmdb.org/t/p/w300/AmUR4UDDuY2t1EhENw5iB46JaNg.jpg
https://image.tmdb.org/t/p/w300/gyIyZOKJ1y80p74GjZuqpfhFy93.jpg
ht

https://image.tmdb.org/t/p/w300/bpBPflXZD1GlNhpYQ5Nu8gftPkG.jpg
https://image.tmdb.org/t/p/w300/k40WFAXMRekWEqsjURO3jiWob67.jpg
https://image.tmdb.org/t/p/w300/jMwDRI4qu5FrSAzf7BfNveuRVf9.jpg
https://image.tmdb.org/t/p/w300/zhiHQIgLSfII3ynPg2LRc1ZBrAE.jpg
https://image.tmdb.org/t/p/w300/hyaOKmxuF8YJKHqWlwpiZU5QsAr.jpg
https://image.tmdb.org/t/p/w300/6lXglg96vfFuHCDdmNof1bdyd0F.jpg
1190 posters saved
https://image.tmdb.org/t/p/w300/zTuqPq1PBsIoG2DV9NgKPgdaoSX.jpg
https://image.tmdb.org/t/p/w300/6lE0EahK7xDOYWRH6On5uKPnwQZ.jpg
https://image.tmdb.org/t/p/w300/yCUahqDKdQt14DqL8fW9LzP2ZMo.jpg
https://image.tmdb.org/t/p/w300/rTCSO24BVkwQWpb0zL6yWeszkaP.jpg
https://image.tmdb.org/t/p/w300/lWCooTiA6sSLdCA4hzToHKKWBZb.jpg
https://image.tmdb.org/t/p/w300/6cZp6bho46sCGKJ3by3hnhCs5TD.jpg
https://image.tmdb.org/t/p/w300/kvlKBGtyyNVyGJB72aD1vwPk2d4.jpg
https://image.tmdb.org/t/p/w300/xi8Pd9YYsHEc53Wkuxz2OBj6Fnq.jpg
https://image.tmdb.org/t/p/w300/r386y8YpePXoS7P4jHWZRTF36Zb.jpg
https://image.tmdb.or

https://image.tmdb.org/t/p/w300/g3XmEqaUx8otzoLEFYSayXs2BmQ.jpg
1310 posters saved
https://image.tmdb.org/t/p/w300/fcOziYVE7iGW3H7tM4yLod4ExvK.jpg
https://image.tmdb.org/t/p/w300/o9lUIJWeSWGtgw9xVH1GtpvwWcI.jpg
https://image.tmdb.org/t/p/w300/reh2SdnPDmFbjWWAEbTv9GgQbke.jpg
https://image.tmdb.org/t/p/w300/2UDsmTvWSxVvuGoRQyV3zVaIRyu.jpg
https://image.tmdb.org/t/p/w300/aPfALBxfgx2tsbTwRMR1MbNHSvn.jpg
https://image.tmdb.org/t/p/w300/A1S3lgrgV8m6AMvEPiX6zGmdCen.jpg
https://image.tmdb.org/t/p/w300/bEIjH6GroeWaBkn6o7MVpMNbuyw.jpg
https://image.tmdb.org/t/p/w300/3CmK3XurcLeUyMifCR28ibzupbB.jpg
https://image.tmdb.org/t/p/w300/fkwkkUnaTmhaWt2OGBRepYr7yB5.jpg
https://image.tmdb.org/t/p/w300/7eBM9T0CKZpy59dIsvAg4GArTg8.jpg
1320 posters saved
https://image.tmdb.org/t/p/w300/eQmBi8qx286i1gMDSLrSbHkYhjy.jpg
https://image.tmdb.org/t/p/w300/aNdLEAK48fjVPk9r6Ei6r90Itgz.jpg
https://image.tmdb.org/t/p/w300/riHE5VDi35sZe0XybM3L4jaErUD.jpg
https://image.tmdb.org/t/p/w300/32NkI9HqueF0THSQFKJVGtnBHsA.jpg
ht

...

https://image.tmdb.org/t/p/w300/btgdJwSL0SDK2vu4vu061EUjVpZ.jpg
https://image.tmdb.org/t/p/w300/sPnH9GAE0hApgj1WrBdA2gGq4wx.jpg
https://image.tmdb.org/t/p/w300/s4gKQhdLXNp006nitMacaw7KFLy.jpg
https://image.tmdb.org/t/p/w300/pJoofuXXnEsDu5U4L1L48qkgUPf.jpg
https://image.tmdb.org/t/p/w300/dy5kKS7qbEHY9Smc2sYByd3D81.jpg
https://image.tmdb.org/t/p/w300/kjEElgEFtAtyadUt8JayBLUw5Sf.jpg
https://image.tmdb.org/t/p/w300/ufE8KAkLQT9u91AgDSc5Oxa3DjW.jpg
https://image.tmdb.org/t/p/w300/xzRRxS91akxrsataS71Mj3riyPR.jpg
https://image.tmdb.org/t/p/w300/819SCRuKsyqYlpCn3fzBWfahWiH.jpg
10200 posters saved
https://image.tmdb.org/t/p/w300/3CHdUrymK5eZvjMRUROvK2qA3fB.jpg
https://image.tmdb.org/t/p/w300/5V676o9WS4rIuJssfMunVMHzxvt.jpg
https://image.tmdb.org/t/p/w300/8YqscC1BUIi9rKn3muFBodyDWqH.jpg
https://image.tmdb.org/t/p/w300/hcCVCCvMsJXPKr5NTSWNnuM5LPf.jpg
https://image.tmdb.org/t/p/w300/kzWRtyMtazWA22LJXA64KKZ3slY.jpg
https://image.tmdb.org/t/p/w300/ywiYYoYZFMxRl2wm4nwFRdUZYHq.jpg
https://image.tmdb.or

https://image.tmdb.org/t/p/w300/mKedM3lkGfqrXfbJdRuaerFwC7z.jpg
https://image.tmdb.org/t/p/w300/jt0nmzkOFPjqsDpDdCo6I12k2E2.jpg
https://image.tmdb.org/t/p/w300/2dXEEznF48Hr48BU9QQeiXVLaOr.jpg
10320 posters saved
https://image.tmdb.org/t/p/w300/cXIZPs9CBUQnl8dyhQaH3QhCYZr.jpg
https://image.tmdb.org/t/p/w300/5IEIy3TsdtashRxNAgI2TQ34o23.jpg
https://image.tmdb.org/t/p/w300/nFQUwWxcjw38V8whGkaKDDn2AQ6.jpg
https://image.tmdb.org/t/p/w300/7tjT7yhwFBLGag2iH2iJCA5c624.jpg
https://image.tmdb.org/t/p/w300/9vWhhubHkFoqthGMI6RxJ01u7TI.jpg
https://image.tmdb.org/t/p/w300/naXjdtDQXK9EeWq2Oy6yP0h2qeT.jpg
https://image.tmdb.org/t/p/w300/q5BmCAnJbISgA7JpCAPIkoyNtwC.jpg
https://image.tmdb.org/t/p/w300/gVx8fBd0iBrlra0bH0RbKRnnV03.jpg
https://image.tmdb.org/t/p/w300/AsWol0XNnQhwSQYeyuOz6uhnVEk.jpg
https://image.tmdb.org/t/p/w300/A3XtobQzLgc1DP66ALXA49l4ate.jpg
10330 posters saved
https://image.tmdb.org/t/p/w300/mGNPlGgCg7BjA7ZYyu5AC05GhqE.jpg
https://image.tmdb.org/t/p/w300/82AvauFnpTwtBxWyCe2MUPJBFKW.jpg


https://image.tmdb.org/t/p/w300/xiQKxPbn3LyVqI3cBtdyp4XIdLx.jpg
https://image.tmdb.org/t/p/w300/urhExTHFeQCE5bEd3yZfbGfxXtV.jpg
https://image.tmdb.org/t/p/w300/iQPN6WBKdfE91qNHu1u5oEOR8CN.jpg
https://image.tmdb.org/t/p/w300/gOp9GoelgUmOLwSRyRsGDg4TDey.jpg
https://image.tmdb.org/t/p/w300/tZ4rkXHdJqp8pVcZ1y6apkoPR9m.jpg
https://image.tmdb.org/t/p/w300/uRKHglLbA1uRp8sjb5KiStb1ldV.jpg
https://image.tmdb.org/t/p/w300/wBzpfRnx7A91mkBsLbRtwYLxNPX.jpg
Poster https://image.tmdb.org/t/p/w300/vKuWQJLUc36f7LefjucjsatVK3a.jpg couldn't be retrieved
https://image.tmdb.org/t/p/w300/rXwi3tEZdcdInq4Zt5y465edRwA.jpg
https://image.tmdb.org/t/p/w300/eArHpgmfZXD6D0uUVYgVfvodZdS.jpg
10450 posters saved
https://image.tmdb.org/t/p/w300/7M3pvjBG9MUR0Bn5ZwqiuBQYinM.jpg
https://image.tmdb.org/t/p/w300/x57ziRgu1S0XipMDxiV8ctVsItJ.jpg
https://image.tmdb.org/t/p/w300/p97gXfE4IBLqqjYXEfrGLezsi3G.jpg
https://image.tmdb.org/t/p/w300/zY6ObzQfCp6RO4ONqP4snfVVSdH.jpg
https://image.tmdb.org/t/p/w300/gGPa1nqFwF80HJxLK8yMoc3

https://image.tmdb.org/t/p/w300/utYuIoB6M82GM8gayVbZFr0zyvd.jpg
https://image.tmdb.org/t/p/w300/uxZxOuQ1JnAR4pnfvVRnCNHGEwg.jpg
https://image.tmdb.org/t/p/w300/iivLrDdvIAyrAkjxkgvJXjMfHwO.jpg
https://image.tmdb.org/t/p/w300/kY9kif6rxvs77l5JhI4zMraVqm.jpg
https://image.tmdb.org/t/p/w300/poUrPvyWpQSnvyoiM42EHuuCPUa.jpg
https://image.tmdb.org/t/p/w300/jC6kjckGjdOLibPVyZpqj8W3Suk.jpg
10570 posters saved
https://image.tmdb.org/t/p/w300/9hE6vesQoSREyvJha9z0g5OryHH.jpg
https://image.tmdb.org/t/p/w300/gwiY5RJFYSjsoSwoeeo6AbGYjpb.jpg
https://image.tmdb.org/t/p/w300/zkbFZIFyWWMLY796G8y7XrtM2L2.jpg
https://image.tmdb.org/t/p/w300/hO5dGO0oNxkqhQlepvvT5oGWXU5.jpg
https://image.tmdb.org/t/p/w300/vy6LGHpwOVULVOqTLnJyZIbiDDi.jpg
https://image.tmdb.org/t/p/w300/5oW5EQxuCNUGP1pukv35mevDfXG.jpg
https://image.tmdb.org/t/p/w300/iE6SvhvGjKnes3p2VNKwvcmTfBU.jpg
https://image.tmdb.org/t/p/w300/5Aeq8Eoj6hWpjI13oVQmqrBULJi.jpg
https://image.tmdb.org/t/p/w300/u6usyT5ovLIhGB2uW5iCgKn6PxI.jpg
https://image.tmdb.or

https://image.tmdb.org/t/p/w300/5uRnjxNX8DCmLyvYiqLfpdaeqEs.jpg
https://image.tmdb.org/t/p/w300/cNdqjhEhaubx3zK9bdXX4z9hWRi.jpg
https://image.tmdb.org/t/p/w300/fyz8ii2vknNk5CEZ49JdNahBuTt.jpg
https://image.tmdb.org/t/p/w300/9wChSuUoQDiBKUnGTcYMkzRha60.jpg
https://image.tmdb.org/t/p/w300/9pzzhnrLuPJXX2JpqSTZHrrbIhd.jpg
https://image.tmdb.org/t/p/w300/8P3nC00mAXTkiT6MrwZqbgZvYVj.jpg
https://image.tmdb.org/t/p/w300/i8i7xHztK5fb4ZXQ6BhOEwmnpQj.jpg
https://image.tmdb.org/t/p/w300/uMQy0ojGlKhhgTZfSZkyThv4W4H.jpg
https://image.tmdb.org/t/p/w300/n1eY8wjsyMkx6O0QukDDsDGJzMf.jpg
https://image.tmdb.org/t/p/w300/g3YT72aO6PCzzHmWgHDLvV0mKL.jpg
10700 posters saved
https://image.tmdb.org/t/p/w300/1iGMCMla4zs3PQ9C0uo9fQrSi0n.jpg
https://image.tmdb.org/t/p/w300/rUV2UBR2bImHIv8klHNa3hDghU4.jpg
https://image.tmdb.org/t/p/w300/2rN88lgz1qX2iRI73tD8XwEo6xv.jpg
https://image.tmdb.org/t/p/w300/xawXqdVDVga9DlhMepIchcuINo1.jpg
https://image.tmdb.org/t/p/w300/klPBNMMbb2fjBV4k7LWn7d5jDlI.jpg
https://image.tmdb.or